In [ ]:
# # Create Python package from Rust code
# maturin develop --release

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import time
import numpy as np

from tigramite.data_processing import DataFrame
from tigramite.pcmci import PCMCI
from tigramite.independence_tests.parcorr import ParCorr

import pcmcirustpy

# Synthetic data generation

In [ ]:
def generate_synthetic_data(
    n_time, n_vars, max_lag, causal_structure, noise_level=0.1, seed=42
):
    """
    Generate synthetic time series data with a known time-lagged causal structure.

    Parameters:
        n_time (int): Number of time steps.
        n_vars (int): Number of variables.
        max_lag (int): Maximum lag to consider.
        causal_structure (dict): Dictionary specifying causal relationships.
            Keys are target variable indices, and values are lists of tuples
            (source_variable, lag, coefficient).
        noise_level (float): Standard deviation of the noise added to each variable.
        seed (int, optional): Random seed for reproducibility.

    Returns:
        np.ndarray: Generated time series data of shape (n_time, n_vars).
    """
    if seed is not None:
        np.random.seed(seed)

    # Initialize the data array
    data = np.zeros((n_time, n_vars))

    # Generate initial random values for the first `max_lag` time steps
    # Inclusion of Gaussian noise, simulating real-world data variability, for testing the robustness of PCMCI

    # Use a burn-in period (e.g., generate additional time steps before the actual data) to allow the system to stabilize and ensure stationarity.
    # As stationarity is an assumption of the PCMCI algorithm
    burn_in = 100  # Generate extra time steps to stabilize the system
    data = np.zeros((n_time + burn_in, n_vars))
    data[:max_lag] = np.random.rand(max_lag, n_vars)

    # Generate data for subsequent time steps
    for t in range(max_lag, n_time):
        for target_var in range(n_vars):
            # Add contributions from causal parents
            value = 0.0
            if target_var in causal_structure:
                for source_var, lag, coeff in causal_structure[target_var]:
                    value += coeff * data[t - lag, source_var]

            # Add noise
            value += np.random.normal(0, noise_level)
            data[t, target_var] = value

    return data


causal_structure = {
    0: [
        (1, 1, 0.3), # var1(t-1) -> var0
        (2, 1, 0.4), # var2(t-1) -> var0
        (3, 2, 0.5), # var3(t-2) -> var0
    ],
    1: [(2, 1, 0.2)],  # var2(t-1) -> var1
    2: [(3, 1, 0.1)],  # var3(t-1) -> var2
}
max_lag = 2
n_vars = 20

data = generate_synthetic_data(
    n_time=100_000,
    n_vars=n_vars,
    max_lag=max_lag,
    noise_level=0.05,
    causal_structure=causal_structure,
)

data.shape

(100100, 20)

# Test stationarity of each time series

In [3]:
# # Test stationarity with Augmented Dickey-Fuller test
# from statsmodels.tsa.stattools import adfuller

# def test_stationarity(data, alpha=0.05):
#     """
#     Test the stationarity of a time series using the Augmented Dickey-Fuller test.

#     Parameters:
#         data (np.ndarray): Time series data.
#         alpha (float): Significance level for the test.

#     Returns:
#         bool: True if the series is stationary, False otherwise.
#     """
#     result = adfuller(data)
#     p_value = result[1]
#     return p_value < alpha

# # Test stationarity for each variable in the dataset
# stationary_results = []
# for i in range(data.shape[1]):
#     stationary = test_stationarity(data[:, i])
#     stationary_results.append(stationary)
# stationary_results

# Set common execution parameters

In [4]:
alpha = 0.05
max_condition_set_size = 3
max_subsets = 1

# Execute PCMCI with the Tigramite implementation

In [12]:
# Create a DataFrame object for tigramite.
df = DataFrame(data, var_names=[f'var{i}' for i in range(n_vars)])

# Set-up the PCMCI object using ParCorr as the conditional independence test.
pcmci_obj = PCMCI(dataframe=df, cond_ind_test=ParCorr())

# Run PCMCI with the maximum lag provided
# max_combinations = 3
start = time.time()
results = pcmci_obj.run_pcmci(
    tau_max=max_lag, 
    pc_alpha=alpha, 
    max_conds_dim=max_condition_set_size,
    max_combinations=max_subsets
)
end = time.time()
print("Time taken:", end - start)

Time taken: 61.940051317214966


In [ ]:
# tgplt.plot_time_series_graph(
#     figsize=(12, 4),
#     val_matrix=results['val_matrix'],
#     graph=results['graph'],
#     var_names=[f'var_{i}' for i in range(20)],
#     link_colorbar_label='MCI',
#     label_fontsize=20,
#     tick_label_size=20
# )

# Execute PCMCI with the Rust implementation

In [21]:
start = time.time()
results_rust = pcmcirustpy.run_pcmci(data, max_lag, alpha, max_condition_set_size, max_subsets)
end = time.time()
print("Time taken:", end - start)

Time taken: 2.530893087387085


# Compare results

In [ ]:
# var1(t-1) -> var0, var2(t-1) -> var0, var3(t-2) -> var0
# var2(t-1) -> var1
# var3(t-1) -> var2

In [14]:
print(results["p_matrix"][1,0,1], results_rust["p_matrix"][1,0,1])

0.0 0.0


In [15]:
print(results["val_matrix"][1,0,1], results_rust["val_matrix"][1,0,1])

0.284713805921323 0.28478974163690207


In [16]:
print(results["val_matrix"][3,0,2], results_rust["val_matrix"][3,0,2])

0.44698031988818565 0.44723973762799


In [17]:
print(results["val_matrix"][2,1,1], results_rust["val_matrix"][2,1,1])

0.19401092134493897 0.1939901940577602


In [18]:
print(results["val_matrix"][4,1,2], results_rust["val_matrix"][4,1,2])

0.0033654309005284894 nan


In [19]:
print(results["val_matrix"][0,2,1], results_rust["val_matrix"][0,2,1])

-0.0034849566118117918 nan


In [20]:
print(results["val_matrix"][3,2,1], results_rust["val_matrix"][3,2,1])

0.09760636765845986 0.09768757201039621


# Cross-compilation

In [ ]:
# rustup target add x86_64-unknown-linux-gnu
# maturin build --release --target x86_64-unknown-linux-gnu -i python3.10
# With <home>/.cargo/config